# Task 1 

In [ ]:
import pandas as pd
import numpy as np
import time

%pylab inline
pylab.rcParams['figure.figsize'] = [10, 6]
pylab.rcParams["patch.force_edgecolor"] = True

import seaborn as sns
sns.set(style='ticks')

### How does the data look like?

The first step is to load all the data and to take a first glance at how the data looks (size, attributes, values etcetera). 

In [ ]:
df = pd.read_csv("/home/marleen/projects/DM2018/ODI-2018.csv")
print(df.shape)
df.head()

It can be observed that there are 217 records and 14 attributes (excluding the timestamp). The attributes have different types, lets discuss shortly all the attributes and their corresponding types:

** 1. What program are you in? **
This was an open question asking for the user's program (categorical). The result should be a (set of) word(s). Note here, that the user could fill in anything they wanted to which makes it difficult to categorize the data. Data cleaning is needed to group the different programs. An example: Artificial Intelligence should belong to the same category as AI. 

** 2. Have you taken a course on machine learning? **
Here, the user could only specify "yes" or "no" (binary), it is therefore relatively easy to analyze this data. No cleaning is needed. There might be unknowns which have to be filtered out.

** 3. Have you taken a course on information retrieval? **
Here, 0 stands for no and 1 stands for yes (binary). The same holds as for attribute 2. 

** 4. Have you taken a course on databases? **
Here, mu stands for yes and sigma stands for no (binary). The same holds as for attribute 2. 

** 5. What is your gender? **
Here a user could specify male, female or unknown (categorical). Note that the unknown again might be difficult for analyzing the data.

** 6. Chocolate makes you.... **
The possible answers here were 'neither', 'I have no idea what you are talking about' or 'fat' (categorical). 

** 7. When is your birthday (date)? **
This was an open question meaning that the user could choose how to write down it's birthday. This makes it difficult to analyze since there are many ways to do so (a better way would be to present a calendar to the user in which it can choose a dat). Data cleaning might become involved here! This is somewhat continuous data. 

** 8. Number of neighbours sitting around you **
Here a user could write anything. Data cleaning is important here. Maybe values are written like words. 

** 9. Did you stand up? **
Possible answers: yes, no and unknown. (Categorical)

** 10. You can get £100... **
Here the user could fill in anything they wanted to. Note that the maximum should be 100 pounds. Data cleaning is really necessary here since some users have valus too high and others used words. (Continuous)

** 11. Give a random number **
Can be any number (or word?) data cleaning necessary. (Continuous)

** 12. Time you went to bed **
Needs cleaning, different users specify time in different ways (AM, PM, words, 10:23, 10.32 etcetera). Continuous.

** 13. What makes a good day for you (1) and (2)? **
User could type anything they want. Cleaning might not be necessary (except for non existing words or numbers maybe). Categorical

### Relationships between attributes 

It is possible to hypothesize which attributes have relationships. For example, a users program (1) probably relates to the courses they have followed (2, 3 and 4). 

Also, gender might tell something about the program a user follows (in some programs it is known that there are more males/females). 

(12) might influence the answers given in (13). 

Also, (8) and (9) might have a relationship. 



### Study Program


Try to categorize study programs in:
- Artificial intelligence
- Business analytics
- Computer Science
- Computational Science
- Bioinformatics
- Econometrics and operation research
- Quantitative Risk Management
- PhD

In [ ]:
# make copy of original dataset
df2 = df.copy()

# remove first row
df2 = df2.drop(df.index[0])

# turn column into type string
df2['What programme are you in?'] = df2['What programme are you in?'].apply(str)

# computational science
searchfor = ['Computational', 'CLS', 'CSL']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'Computational Science'

# computer science
searchfor = ['Computer', 'Big data engineering']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'Computer Science'
df2['What programme are you in?']=df2['What programme are you in?'].replace('cs', 'Computer Science')
df2['What programme are you in?']=df2['What programme are you in?'].replace('CS', 'Computer Science')

# artificial science
searchfor = ['A. I.', 'Ai', 'Artificial', 'Intelligence']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'Artificial Intelligence'

# phds
searchfor = ['phd']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'PhD'

# Quantitative Risk Management
searchfor = ['QRM', 'Duisenberg', 'Risk management']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'Quantitative Risk Management'

# Bioinformatics
searchfor = ['Bioinformatics', 'Biology', 'bio']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'Bioinformatics'
df2['What programme are you in?']=df2['What programme are you in?'].replace('Bioinformatcis', 'Bioinformatics')

# Business Analytics
searchfor = ['Business Analytics']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'Business Analytics'
df2['What programme are you in?']=df2['What programme are you in?'].replace('BA', 'Business Analytics')

# Econometrics & Operations Research
searchfor = ['EOR', 'Operations']
df2.loc[df2['What programme are you in?'].str.contains('|'.join(searchfor), case=False), \
        'What programme are you in?'] = 'Econometrics & Operations Research'
df2['What programme are you in?']=df2['What programme are you in?'].replace('OR', 'Econometrics & Operations Research')

# Econometrics
df2['What programme are you in?']=df2['What programme are you in?'].replace('Econometrics ', 'Econometrics')

# Set programs with < 2 people to 'Other' category
value_counts = df2['What programme are you in?'].value_counts()
to_remove = value_counts[value_counts <= 2].index
df2['What programme are you in?'].replace(to_remove, 'Other', inplace=True)

# make extra column containing labels
df2["What programme are you in?"] = df2["What programme are you in?"].astype('category')
df2["programme_cat"] = df2["What programme are you in?"].cat.codes

### Cleaning Dates

This is quite involved, since there are many ways in which the date is provided. 

In [ ]:
# change column name and type
df3 = df2.copy()
df3 = df3.rename(columns={'When is your birthday (date)?': 'birthday'})
# df3[['birthday']]=df3[['birthday']].astype(str)

# necessary replacements
replace_dict = {"/":"-", " ":"-", " ":"-", ".":"-"} 
for initial, replace in replace_dict.items():
    df3['birthday']=df3['birthday'].str.replace(initial, replace)

# replace items without '-' by np.nan
for item in df3['birthday']:
    if ("-" in item)==False:
#         df3['birthday'][df3['birthday'] == item] = "NaN"
        df3['birthday'].loc[df3['birthday'] == item] = "NaN"
        
# change words into numbers for months
replace_dict = {"january":"1", "february":"2", "march":"3", "april":"4", "may":"5",\
                "june":"6", "july":"7", "august":"8", "september":"9", "october":"10",\
               "november":"11", "december":"12"} 
for initial, replace in replace_dict.items():
    df3['birthday']=df3['birthday'].str.replace(initial, replace, case=False)

# set all dates to day-month format
j=0
for item in df3['birthday']:
    itemcopy=item
    date = item.split('-')
    
    # check if still words in dates, remove
    try:
        for i in range(len(date)):
            int(date[i])
    except:
        df3['birthday'].iloc[j] = "NaN"
        item="NaN"
        j+=1
        continue
    
    # remove birth years
    try:
        tempdate=date.copy()
        for i in range(len(date)):
            if int(date[i]) > 31:
                tempdate.remove(date[i])
                newitem = "-".join(tempdate)
                df3['birthday'].iloc[j]=newitem
                item = newitem
    except ValueError as e:
        df3['birthday'].iloc[j] = "NaN"
        item="NaN"
        j+=1
        continue

    # put day and month in correct order
    try:
        date=item.split("-")
        if int(date[0])>12 and int(date[1])>12:
            raise Exception        
        elif int(date[1])>12:
            month=date[0]
            day=date[1]
            newitem=[day,month]
            df3['birthday'].iloc[j]="-".join(newitem)
            item="-".join(newitem)
    except (Exception, ValueError) as e:
        df3['birthday'].iloc[j]="NaN"
        item="NaN"
        j+=1
        continue   
            
    # remove zeroes
    try:
        date=item.split('-')
        date[0]=str(int(date[0]))
        date[1]=str(int(date[1]))
        df3['birthday'].iloc[j]="-".join(date)
        item="-".join(date)
        
    except:
        df3['birthday'].iloc[j] = "NaN"
        item="NaN"
        j+=1
        continue
        
    # remove days
    try:
        int(item.split("-")[1])
        df3['birthday'].iloc[j] = (item.split("-")[1])
        j+=1
    except:
        df3['birthday'].iloc[j] = "NaN"
        item="NaN"
        j+=1
        continue

df3['birthday']=df3['birthday'].astype(float)

### Cleaning Random Numbers

This is easier, we will only consider numbers between 0 and 10. 

In [ ]:
# change column name and type
df4 = df3.copy()
df4 = df4.rename(columns={'Give a random number': 'random_number'})

# iterate over all random numbers
for item in df4['random_number']:
    try:
        newitem=float(item)
        
        # remove everything larger than 10
        if newitem > 10:
            raise Exception
        elif newitem < 0:
            raise Exception
    except:
        df4['random_number'][df4['random_number']==item]="NaN"
df4['random_number']=df4['random_number'].astype(float)


### Clean Greedyness Column

In [ ]:
# change column name and type
df5 = df4.copy()
df5 = df5.rename(columns={'You can get £100 if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would get then? ': 'greedyness'})
df5[['greedyness']]=df5[['greedyness']].astype(str)

# remove pounds stuff
remove_dict=({'£':"", "pond":"","euro":"","pound":"","€ ?:)":"","euros":""})
for initial, replace in remove_dict.items():
    try:
        df5['greedyness']=df5['greedyness'].str.replace(initial, replace)
    except:
        x=1

# greedyness needs to be between 0 and 100
df5['greedyness']=pd.to_numeric(df5['greedyness'], errors='coerce')
mask = ((df5.greedyness > 100)| (df5.greedyness < 0) )
# print(mask)
column_name = 'greedyness'
df5.loc[mask, column_name] = np.nan

#turn all values into cents and make integers of it
df5.greedyness=df5.greedyness.dropna() * 100
df5.greedyness=df5.greedyness.dropna().astype(int)
df5.head()

### Clean Gender Column

In [ ]:
# change column name and type
df6 = df5.copy()
df6 = df6.rename(columns={'What is your gender?': 'gender'})

# replacements
replace_dict=({'male':1,'female':0,'unknown':"NaN"})
for initial, new in replace_dict.items():
    df6.gender=df6.gender.replace(initial, new)

### Clean Courses Columns

In [ ]:
df7 = df6.copy()
df7 = df7.rename(columns={'Have you taken a course on machine learning?': 'machine_learning'})
df7 = df7.rename(columns={'Have you taken a course on information retrieval?': 'info_retrieval'})
df7 = df7.rename(columns={'Have you taken a course on databases?': 'databases'})
df7 = df7.rename(columns={'Have you taken a course on statistics?': 'statistics'})

# replacements for machine learning
replace_dict=({'yes':1,'no':0,'unknown':"NaN"})
for initial, new in replace_dict.items():
    df7.machine_learning=df7.machine_learning.replace(initial, new)
    
# replacements for info retrieval
replace_dict=({'unknown':"NaN"})
for initial, new in replace_dict.items():
    df7.info_retrieval=df7.info_retrieval.replace(initial, new)

# replacements for databases
replace_dict=({'ja':1,'nee':0,'unknown':"NaN"})
for initial, new in replace_dict.items():
    df7.databases=df7.databases.replace(initial, new)

# replacements for statistics
replace_dict=({'mu':1,'sigma':0,'unknown':"NaN"})
for initial, new in replace_dict.items():
    df7.statistics=df7.statistics.replace(initial, new)

# make integers of all courses
df7.statistics = df7.statistics.astype(float)
df7.info_retrieval = df7.info_retrieval.astype(float)
df7.machine_learning = df7.machine_learning.astype(float)
df7.databases = df7.databases.astype(float)

df7.head()

### Length of final columns

In [ ]:
df8=df7.copy()
df8 = df8.rename(columns={'What makes a good day for you (1)?': 'good_day_1'})
df8 = df8.rename(columns={'What makes a good day for you (2)?': 'good_day_2'})
df8['length_answers'] = 0
df8.length_answers.astype(int)
for i in range(1,len(df8.good_day_1) + 1):
    df8.loc[i,('length_answers')] = len(df8.loc[i, 'good_day_1'])+len(df8.loc[i, 'good_day_2'])

## Some nice figures, distributions and plots from the dataset


In [ ]:
df2['What programme are you in?'].value_counts().plot(kind='bar')

In [ ]:
df3.hist(column='birthday', bins=np.arange(0.5,13.5,1))

In [ ]:
df4.hist(column='random_number', bins=np.arange(-0.5,10.5,1))

In [ ]:
# y-axis in bold
rc('font', weight='bold')
 
# Values of each group
study_groups_male=list(df4['What programme are you in?'][df4['What is your gender?']=="male"].value_counts().sort_index())
study_groups_female=list(df4['What programme are you in?'][df4['What is your gender?']=="female"].value_counts().sort_index())
print(df4['What programme are you in?'][df4['What is your gender?']=="male"].value_counts().sort_index())
print(df4['What programme are you in?'][df4['What is your gender?']=="female"].value_counts().sort_index())
bars = [x + y for x, y in zip(study_groups_male, study_groups_female)]
 
# position of the bars 
r = [1,2,3,4,5,6,7,8,9,10]
 
# names of group and bar width
names = ['Artificial Intelligence','Bioinformatics','Business Analytics','Computational Science',\
         'Computer Science','Econometrics','Econometrics & Operations Research',\
         'Other','PhD','Quantitative Risk Management']
barWidth = 1
 
# ticks
plt.xticks(rotation=82)
plt.xticks(r, names, fontweight='bold')
plt.xlabel("group")
 
# Show graphic
plt.show()

In [ ]:
subset = df5[['greedyness','What programme are you in?','What is your gender?']]
value_counts = df5['What programme are you in?'].value_counts()
to_remove = value_counts[value_counts <= 20].index

# subset['What programme are you in?'].replace(to_remove, "NaN", inplace=True)
subset['What programme are you in?'].replace(to_remove, "NaN", inplace=True)
subset = subset[subset['What programme are you in?'] != 'NaN']

# subset['What programme are you in?'] = subset.programme_cat.astype(float)
subset = subset.dropna()
fig, ax = pyplot.subplots(figsize=(7,4))

# sns.boxplot(x=df5.greedyness / 100,y=df5['What programme are you in?'])
sns.boxplot(ax=ax,x=subset.greedyness / 100,y=subset['What is your gender?'])

In [ ]:
# plotting by seaborn
sns.boxplot(x=df5.random_number,y=df5['What programme are you in?'])

In [ ]:
# plotting by seaborn
sns.boxplot(x=df5.random_number,y=df5['What is your gender?'])

In [ ]:
# plotting by seaborn
sns.boxplot(x=df8.length_answers,y=df5['What programme are you in?'])

In [ ]:
# plot % of people who followed a course for each program

# total number of people in programme
totals = np.array(df8['What programme are you in?'].value_counts().sort_index())

# fraction of people following courses per programme
statistics = np.array(df8['What programme are you in?'][df8['statistics']==1].value_counts().sort_index())*100/totals
databases = np.array(df8['What programme are you in?'][df8['databases']==1].value_counts().sort_index())*100/totals
machine_learning = np.array(df8['What programme are you in?'][df8['machine_learning']==1].value_counts().sort_index())*100/totals
info_retrieval = np.array(df8['What programme are you in?'][df8['info_retrieval']==1].value_counts().sort_index())*100/totals

statistics = statistics[0:5]
databases = databases[0:5]
machine_learning = machine_learning[0:5]
info_retrieval = info_retrieval[0:5]

# y-axis in bold
rc('font', weight='bold')
 
# The position of the bars on the x-axis
# r = [1,2,3,4,5,6,7,8,9,10]
r = [1,2,3,4,5]
 
# Names of group and bar width
# names = ['Artificial Intelligence','Bioinformatics','Business Analytics','Computational Science',\
#          'Computer Science','Econometrics','Econometrics & Operations Research',\
#          'Other','PhD','Quantitative Risk Management']
# names = ['AI','BioI','BA','CLS','CS','ECO','EOR',\
#          'Other','PhD','QRM']
names = ['AI','BioI','BA','CLS','CS']
barWidth = 1

fig = plt.figure()
ax = fig.add_subplot(111)    
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

# turn off axis lines and ticks of the big subplot
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')

ax.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')
# ax1.tick_params(labelcolor='w')
# ax2.tick_params(labelcolor='w')

ax3.set_xticks(r)
ax3.set_xticklabels(names, rotation=90)
ax4.set_xticks(r)
ax4.set_xticklabels(names, rotation=90)

names = ["","","","",""]
ax1.set_xticks(r)
ax1.set_xticklabels(names)
ax2.set_xticks(r)
ax2.set_xticklabels(names)

ax1.bar(r, statistics, color='#0e6655', edgecolor='white', width=barWidth)
ax2.bar(r, machine_learning, color='#0e6655', edgecolor='white', width=barWidth)
ax3.bar(r, databases, color='#0e6655', edgecolor='white', width=barWidth)
ax4.bar(r, info_retrieval, color='#0e6655', edgecolor='white', width=barWidth)

# Set common labels
ax.set_xlabel('Programme')
ax.set_ylabel('Percentage of Students (%)')

ax1.set_title('Statistics')
ax2.set_title('Machine Learning')
ax3.set_title('Databases')
ax4.set_title('Information Retrieval')
# plt.tight_layout()
plt.savefig('common_labels.png', dpi=300)

## Predicting study program based on courses followed

In [ ]:
df8.programme_cat = df8.programme_cat.astype(int)
df8.dtypes

In [ ]:
# drop programmes with <20 students
value_counts = df8['programme_cat'].value_counts()
# print(value_counts)
# to_remove = value_counts[value_counts <= 20].index
# df8.replace(to_remove, np.nan, inplace=True)
# df8['What programme are you in?'].drop(to_remove)

feature_names = ['statistics', 'databases', 'info_retrieval', 'machine_learning']
keep = value_counts[value_counts >= 20].index
subset=df8[feature_names+['programme_cat']].dropna()

# part_set = df8[]
X = subset[feature_names]
y = subset['programme_cat'].astype(int64)

In [ ]:
# imports 
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# scale attribute values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
logreg = LogisticRegression()
model = logreg.fit(X_train, y_train)
# model = logreg

print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(model.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(model.score(X_test, y_test)))

In [ ]:
# Perform 6-fold cross validation
scores = cross_val_score(model, X, y, cv=6)
print ("Cross-validated scores:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
clf = DecisionTreeClassifier().fit(X_train, y_train)
model=clf
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

# Perform 6-fold cross validation
scores = cross_val_score(model, X, y, cv=6)
print ("Cross-validated scores:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
model = gnb.fit(X_train, y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))

# Perform 6-fold cross validation
scores = cross_val_score(model, X, y, cv=6)
print ("Cross-validated scores:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Greedyness based on program and gender

In [ ]:
# above this treshold, you are greedy
greedy_treshold = 150

# create new column with 1 begin greedy 0 not greedy
df7['greedy_indicator']=np.where(df7['greedyness']>=greedy_treshold, 1, 0)

feature_names = ['programme_cat','gender']
subset=df7[feature_names+['greedy_indicator']].dropna()


subset.gender=subset.gender.astype(float)
subset=subset.dropna()

# remove small programmes
value_counts = subset['programme_cat'].value_counts()
to_remove = value_counts[value_counts <= 20].index
subset['programme_cat'].replace(to_remove, "NaN", inplace=True)
subset['programme_cat'] = subset.programme_cat.astype(float)
subset = subset.dropna()
# keep = value_counts[value_counts >= 20].index

X = subset[feature_names]
y = subset['greedy_indicator'].astype(int64)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# scale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logreg = LogisticRegression()
model = logreg.fit(X_train, y_train)

# model = DecisionTreeClassifier()
# model = KNeighborsClassifier()
# model = gnb.fit(X_train, y_train)
# model = svm.fit(X_train, y_train)

# Perform 6-fold cross validation
scores = cross_val_score(model, X, y, cv=4)
print ("Cross-validated scores:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# subset['multiplied']
subset.greedy_indicator.value_counts()